In [1]:
import numpy as np
import pandas as pd
import zipfile


c:\Users\Matias\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Matias\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [1]:
#pip install implicit

     ------------------------------------ 752.1/752.1 kB 312.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
zip_path = '../00_Data_Bases/Cluster5_1_items.zip' 
csv_filename = 'Cluster5_1_items.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(csv_filename) as f:
        df = pd.read_csv(f)

### Matriz de Interaccion


In [3]:
df["score"] = df["reordered"] * 2 + 1 + (df["days_since_prior_order"] <= 7).astype(int)


In [4]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

# Crear mapeo de IDs a índices numéricos
user_map = {u: i for i, u in enumerate(df["user_id"].unique())}
item_map = {i: j for j, i in enumerate(df["product_id"].unique())}

df["user_index"] = df["user_id"].map(user_map)
df["item_index"] = df["product_id"].map(item_map)

# Crear la matriz dispersa usuario-producto
user_item_matrix = csr_matrix((df["score"], (df["user_index"], df["item_index"])))



In [5]:
# Revisar si los índices están dentro del rango esperado
max_user_idx = max(user_map.values())
max_item_idx = max(item_map.values())

print(f"Índice máximo de usuario: {max_user_idx}, Total usuarios en matriz: {user_item_matrix.shape[0]}")
print(f"Índice máximo de producto: {max_item_idx}, Total productos en matriz: {user_item_matrix.shape[1]}")


Índice máximo de usuario: 37972, Total usuarios en matriz: 37973
Índice máximo de producto: 133, Total productos en matriz: 134


In [8]:
# Configurar y entrenar el modelo ALS
model = AlternatingLeastSquares(factors=200, regularization=0.1, iterations=500)
model.fit(user_item_matrix.T)

c:\Users\Matias\anaconda3\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0029976367950439453 seconds
  warnings.warn(


  0%|          | 0/500 [00:00<?, ?it/s]

In [9]:
# Seleccionar un usuario con más compras (el segundo más frecuente en este caso)
user_id = df["user_id"].value_counts().index[1]  
user_idx = user_map.get(user_id)  # Convertir user_id a user_index

# Crear un diccionario inverso para mapear índices internos a product_id
reverse_item_map = {v: k for k, v in item_map.items()}

# Verificar que el usuario existe en el modelo y tiene datos en la matriz
if user_idx is not None and user_idx < model.user_factors.shape[0]:
    if user_item_matrix[user_idx].nnz > 0:  # Verifica que el usuario tiene interacciones
        # Generar recomendaciones
        item_indices, scores = model.recommend(user_idx, user_item_matrix[user_idx], N=5)

        # Convertir índices internos a product_id
        recommended_items = [reverse_item_map[i] for i in item_indices if i in reverse_item_map]

        print(f"Recomendaciones para el usuario {user_id}: {recommended_items}")
    else:
        print(f"El usuario {user_id} no tiene suficientes interacciones en la matriz.")
else:
    print(f"El usuario {user_id} no tiene datos suficientes en el modelo.")


El usuario 177197 no tiene datos suficientes en el modelo.
